## Install Kaggle dependecies

In [0]:
!pip install kaggle azure-storage-blob

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Import libraries

In [0]:
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi
from azure.storage.blob import BlobServiceClient
from pyspark.dbutils import DBUtils
from azure.storage.blob import BlobServiceClient
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import col, expr

## Downloading raw file from Kaggle and puttting to Azure datalake

In [0]:
# Cell 1: Import necessary libraries
import os
from kaggle.api.kaggle_api_extended import KaggleApi
from pyspark.sql.functions import col, year, month, dayofmonth, current_date

# Cell 2: Configurations
KAGGLE_USERNAME = KAGGLE_USERNAME  
KAGGLE_KEY = KAGGLE_KEY
, #This located under Github Repository secretsAZURE_STORAGE_ACCOUNT_NAME = noviprojectst'
AZURE_STORAGE_ACCOUNT_KEY = AZURE_STORAGE_ACCOUNT_KEY,  #This located under Github Repository secretsCONTAINER_NAME = 'incoming'
DATASET_PATH = 'jayaantanaath/student-habits-vs-academic-performance'

# Set Kaggle credentials as environment variables
os.environ['KAGGLE_USERNAME'] = KAGGLE_USERNAME
os.environ['KAGGLE_KEY'] = KAGGLE_KEY

# Authenticate with Kaggle
api = KaggleApi()
api.authenticate()

# Download dataset
api.dataset_download_files(DATASET_PATH, path='/dbfs/tmp/data', unzip=True)

# Initialize Spark session
spark = SparkSession.builder.appName("KaggleDataToParquet").getOrCreate()

# Set Spark properties to configure Azure credentials
spark.conf.set(f"fs.azure.account.key.{AZURE_STORAGE_ACCOUNT_NAME}.blob.core.windows.net", AZURE_STORAGE_ACCOUNT_KEY)

# Read the dataset into a Spark DataFrame
csv_file_path = '/dbfs/tmp/data/Student_Habits_and_Academic_Performance.csv'
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

# Add current year, month, and day columns
df = df.withColumn("year", year(current_date())) \
       .withColumn("month", month(current_date())) \
       .withColumn("day", dayofmonth(current_date()))

# Write the DataFrame as a Parquet file
output_path = f"wasbs://{CONTAINER_NAME}@{AZURE_STORAGE_ACCOUNT_NAME}.blob.core.windows.net/parquet_output"
df.write.partitionBy("year", "month", "day").parquet(output_path)

print("Dataset written as Parquet file with partitions successfully!")

Dataset written as Parquet file with partitions successfully!
